In [ ]:
import sys
# add parent directory to sys.path so that python finds the modules
sys.path.append('..')

import os
import math

import psycopg2
from postgis.psycopg import register
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

from db_importer.settings import *

class DatabaseConnection(object):
    def __enter__(self):
        self.conn = psycopg2.connect(f"dbname='{DB_NAME}' user='{DB_USER}' password='{DB_PASSWORD}' host='{DB_HOST}' port='{DB_PORT}'")
        self.conn.autocommit = True

        register(self.conn)
        self.cur = self.conn.cursor()
 
        return self.cur

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_tb is None:
            self.conn.commit()
            self.cur.close()
            self.conn.close()
        else:
            self.conn.rollback()
            self.cur.close()
            self.conn.close()

In [ ]:
from shapely.geometry import Point
from shapely.geometry import box

start_rect = box(13.416354, 52.521752, 13.416705, 52.521961)
end_rect = box(13.416912, 52.522261, 13.417263, 52.52247)

def crop_intersection_SimRa(x):
    mask_first = x.coords.apply(lambda x: start_rect.contains(Point(x)))
    mask_end = x.coords.apply(lambda x: end_rect.contains(Point(x)))
    try:
        first = mask_first[mask_first==True].index[0]
        last = mask_end[mask_end==True].index[-1]
        return x.loc[first:last]
    except:
        print('check')
        
def to_float_arr(x):
    res = []
    for c in x:
        res.append(float(c))
    return res


In [ ]:
ped_files = ['VM2_-2112701535', 'VM2_-217686115','VM2_-1247665811','VM2_-104300786', 'VM2_330973206', 'VM2_1100569031', 'VM2_421371629', 'VM2_421371629', 'VM2_1476499235', 'VM2_-1451152685', 'VM2_-1523872256']


with DatabaseConnection() as cur:
    cur.execute("""
SELECT filename,
       json_array_elements_text(st_asgeojson(geom_raw) :: json -> 'coordinates') AS coordinates,
       unnest(timestamps)                             timestamps,
       unnest(velos)                                  velos
FROM ride
where st_intersects(geom, st_setsrid(
        st_makebox2d(st_makepoint(13.416354, 52.521752), st_makepoint(13.416705, 52.521961)), 4326))
  AND st_intersects(geom, st_setsrid(
        st_makebox2d(st_makepoint(13.416912, 52.522261), st_makepoint(13.417263, 52.52247)), 4326))
  AND timestamps[1] < '2021-01-01 00:00:00'""")
    res = cur.fetchall()
    df = pd.DataFrame(res, columns=['filename', 'coords', 'timestamps', 'velos'])
    df = df[~df.coords.isnull()]
    df = df[~df['filename'].isin(ped_files)]
    df.coords = df.coords.apply(lambda x: x.replace('[', '').replace(']', '').split(','))
    df.coords = df.coords.apply(lambda x: to_float_arr(x))
    
    grouped = df.groupby('filename', as_index=False)
    df_simra = grouped.apply(lambda x: crop_intersection_SimRa(x))


In [ ]:
from matplotlib import rc

map_data = plt.figure(figsize=(7,7))

rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 16})
rc('text', usetex=True)

colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c']

def plot_location(x, color, alpha):        
    lat, lon = zip(*x.coords.values)
    plt.plot(lon, lat, c=color, alpha=alpha)
    
df_simra.groupby('filename').apply(lambda x: plot_location(x, colors[0], 0.3))

xmin, xmax, ymin, ymax = plt.axis()
plt.xticks([xmin, xmax], [round(xmin, 3), round(xmax, 3)])
plt.xlabel('Longitude in decimal degrees')
plt.yticks([ymin, ymax], [round(ymin, 3), round(ymax, 3)])
plt.ylabel('Latitude in decimal degrees')
plt.plot([], c=colors[0], label='SimRa')
plt.legend(loc='upper right')
plt.savefig('alex_accels_test.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
df_simra

In [ ]:
df_simra['lat'], df_simra['lon'] = zip(*df_simra.coords.values)


fig, ax = plt.subplots(figsize=(10,10))

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velos.min(), df_simra.velos.max())

df_simra_grouped = df_simra.groupby('filename')
for ride_group_name in df_simra_grouped.groups:
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lat, df_ride_group.lon]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments, cmap='viridis', norm=norm)
    # Set the values used for colormapping
    lc.set_array(df_ride_group.velos.values)
    lc.set_linewidth(2)
    line = ax.add_collection(lc)
    # fig.colorbar(lc, ax=ax)

ax.set_xlim(min(df_simra.lat), max(df_simra.lat))
ax.set_ylim(min(df_simra.lon), max(df_simra.lon))
plt.show()

In [ ]:
pd.Series((1, 2, 4))